In [ ]:
%% Code template for calibrating TubeOD or ClampOD and plotting growth curves.

%% NOTE: This template is written in Matlab, so make sure you create a Matlab kernel 
%%in jupyter or copy and paste this code into a MATLAB window before running anything.

%% here is a resource for getting familiar with jupyter and creating a kernel: 
%% https://portal.geomar.de/documents/18749/1308328/2018-09-27_Matlab+Kernel+for+Jupyter+Notebooks.pdf/ecd33b0c-2f3d-49ca-8146-1b957a68597d

In [ ]:
%% load data and variables

%% NOTE: Make sure all datasets and code are in the same folder, 
%% or the path for the data is in the code. Otherwise the code 
%% will not be able to find your data. 

%% use readtable() and table2array() to upload .csv files. There are also options for .txt, .xlsx etc. 

Data = table2array(readtable([Your data path here])

%ClampCalV = data from Labjack, multimeter, potentiostat, or other voltage measurement device.
%ClampCalOD = data from another OD measuring device or method.

In [1]:
%% Calibrating the ClampOD or TubeOD can be done online or by inspection with a multimeter

%% For online calibrations, while sensor is still detecting light shifts,
%% 1. dilute the cell suspension or culture by a known amount at set intervals. 
%% 1.5 Save this file as a .csv and upload it above.
%% 2. Measure the OD of each diluted cell suspension with a benchtop spectrophotometer, 
%% or another OD measurement technique. 

%% Separate the data at the dilution intervals, then find the median voltage reading. 
%% This is your calibration point for that OD.

%This data is not loaded here, but use this for loop as an example of how to separate the voltage readings.
for i = 1: length(ClampCalV)
    if ClampCalV(i,1) > time1 && ClampCalV(i,1) < time2
        ClampCalV(i,5) = 1;
    elseif ClampCalV(i,1) > time3 && ClampCalV(i,1) < time4
        ClampCalV(i,5) = 2;
    elseif ClampCalV(i,1) > time5 && ClampCalV(i,1) < time6
        ClampCalV(i,5) = 3;
    elseif ClampCalV(i,1) > time7 && ClampCalV(i,1) < time8
        ClampCalV(i,5) = 4;
    elseif ClampCalV(i,1) > time9 && ClampCalV(i,1) < time10
        ClampCalV(i,5) = 5;
        
    else
        ClampCalV(i,5) = 0;
    end
    
end

% Calibrating voltage data by finding the median value at each interval.

for j = 1:5
    cal_VData(j) = median(ClampCalV(find(ClampCalV(:,5) == j),2));
    
end

In [ ]:
%% use the fit() function to calculate first and second order polynomial fits, where -log(V) is the x value
%% and OD is the y-value. You can also use this function to calculate r-square values and root mean squared 
%% error for each curve. 

%Note, this data is not loaded, so only use these lines as a guide for your own data.
[curve1, goodness1, output1] = fit(-log10(ClampCalV),ClampCalOD, 'poly1', 'normalize', 'on');
[curve2, goodness2, output2] = fit(-log10(ClampCalV),ClampCalOD, 'poly2', 'normalize', 'on');

rsquare1 = goodness1.rsquare;
rmse1 = goodness1.rmse;
rsquare2 = goodness2.rsquare;
rmse2 = goodness2.rmse;

%% (optional) find median values from growth curve voltage data to reduce noise OD measurements

medVolt = zeros(length(Data),1);

%convert to relative time and from s to hours
time_hr = zeros(length(Data),1);
time_relative = (Data(:,1)-Data(1,1))./3600; 
all_time = (Data(:,1)-Data(1,1))./3600;

%calculate OD values from formula, use only median of every 25 readings
for i = 25:length(Data)
    if (i/25) == round(i/25)
        medVolt(i) = median(Data([i-24:i],2));
        time_hr(i) = time_relative(i,1);
    end
    
end
time_hr = time_hr(any(time_hr,2),:);
medVolt = medVolt(any(medVolt,2),:);

%% use the calibration functions to calculate OD values from each median voltage value. 
%% together with time, these values make up the growth curve. 
OD1fit = curve1(-log10(medVolt));
OD2fit = curve2(-log10(medVolt));

%% plot voltage data vs. time

figure("Name", "Voltage data")

hold on

scatter(all_time, Data)

ylabel("Voltage (V)")
xlabel("Time (h)")

%% plot calibration, -log10(voltage) vs. OD

figure("Name", "Calibration")

hold on
scatter(-log10(ClampCalV), ClampCalOD, 'filled');
p1 = plot(curve1, ':');
p2 = plot(curve2, '-');


%% plot growth curve in OD vs. time

figure("Name", "growth curve")

hold on
% Add concurrent manual calibration points if available
p4 = scatter((time_OD_hr), ManualCal(:,4), [], mapGrey(9,:), 'filled');
p1 = plot(time_hr, OD1fit, 'LineWidth', 2);
p2 = plot(time_hr,OD2fit, 'LineWidth', 2);


%% Calculate growth rates when cells are in log phase

figure("Name", "growth rate")

hold on
plot(time_hr, log2(OD1fit), 'LineWidth', 2);
plot(time_hr, log2(OD2fit), ':', 'LineWidth', 2);
gr_1 = fit(time_hr(logical((time_hr >= 1).*(time_hr <= 4))), log2(OD1fit(logical((time_hr >= 1).*(time_hr <= 4)))), 'poly1');
gr_2 = fit(time_hr(logical((time_hr >= 1).*(time_hr <= 4))), log2(OD2fit(logical((time_hr >= 1).*(time_hr <= 4)))), 'poly1');

%Compare the growth rates using the 1st and 2nd order polynomial fits. 
plot(gr_1)
plot(gr_2)
xlabel('Time (h)')
ylabel('log2(OD)')